In [2]:
from astronomaly.data_management import image_reader
from astronomaly.preprocessing import image_preprocessing
from astronomaly.feature_extraction import power_spectrum, wavelet_features, autoencoder
from astronomaly.dimensionality_reduction import decomposition
from astronomaly.anomaly_detection import isolation_forest, human_loop_learning
from astronomaly.postprocessing import scaling
from astronomaly.utils import utils

import importlib
import matplotlib.pyplot as plt
import pywt
import numpy as np
import scipy
from sklearn.manifold import TSNE
import pandas as pd
%matplotlib widget

In [3]:
importlib.reload(image_reader)
importlib.reload(image_preprocessing)
importlib.reload(power_spectrum)
importlib.reload(wavelet_features)
importlib.reload(autoencoder)
importlib.reload(decomposition)
importlib.reload(scaling)
importlib.reload(isolation_forest)
importlib.reload(human_loop_learning)
importlib.reload(utils)

Failed to import Keras. Deep learning will be unavailable.


<module 'astronomaly.utils.utils' from '/home/michelle/Project/Anomaly/astronomaly/astronomaly/utils/utils.py'>

In [4]:
fitsfile = '/home/michelle/BigData/Anomaly/ds9_slice1.fits'

astro_img = image_reader.AstroImage(fitsfile)

Reading image data from /home/michelle/BigData/Anomaly/ds9_slice1.fits...
Done!


In [5]:
print(astro_img.name)

ds9_slice1


In [6]:
# plt.figure()
# plt.imshow(astro_img.image)

In [7]:
x0 = 3801
y0 = 3176


delt = 64
cutout = astro_img.image[y0-delt:y0+delt,x0-delt:x0+delt]
cutout = image_preprocessing.image_transform_log(cutout)


x0 = 3801+20
y0 = 3176+20

cutout2 = astro_img.image[y0-delt:y0+delt,x0-delt:x0+delt]
cutout2 = image_preprocessing.image_transform_log(cutout2)

plt.figure()
plt.subplot(121)
plt.imshow(cutout, origin='lower')
plt.subplot(122)
plt.imshow(cutout2, origin='lower')
plt.tight_layout()

print(astro_img.coords[x0,0], astro_img.coords[y0,1])

FigureCanvasNbAgg()

60.888272842008675 -80.54455710658605


In [8]:
pipeline_dict = image_reader.read_images('/home/michelle/BigData/Anomaly/')

Reading image data from /home/michelle/BigData/Anomaly/ds9_slice1.fits...
Done!


In [9]:
pipeline_dict.keys()

dict_keys(['images'])

In [10]:
pipeline_dict = image_preprocessing.generate_cutouts(pipeline_dict, window_size=128, window_shift=128, transform_function=image_preprocessing.image_transform_log)

Generating cutouts...
Done!


In [11]:
current_ind = 0

def onkeypress(event):
    global current_ind
    plt.gcf()
    if event.key=='right' and current_ind<len(df):
        current_ind += 1

    elif event.key=='left' and current_ind>0:
        current_ind -= 1
    fact = 2
    #toplot = img[x0-window_size*fact:x0+window_size*fact, y0-window_size*fact:y0+window_size*fact]
    plt.clf()
    event.canvas.figure.gca().imshow(pipeline_dict['cutouts'][current_ind])

    plt.title(current_ind)
    event.canvas.draw()

fig = plt.figure()
fig.canvas.mpl_connect('key_press_event', onkeypress)

#plt.imshow(np.log(cutouts[ind]+np.abs(1.1*np.min(cutouts[ind]))))
plt.imshow(pipeline_dict['cutouts'][current_ind])
plt.title(current_ind)

FigureCanvasNbAgg()

Text(0.5, 1.0, '0')

In [12]:
pipeline_dict = power_spectrum.extract_features_psd2d(pipeline_dict, nbins='auto')

Extracting 2D PSD...
Done!


In [12]:
pipeline_dict = wavelet_features.extract_features_wavelets(pipeline_dict)

Extracting wavelets...
Done!


In [13]:
pipeline_dict = autoencoder.extract_features_autoencoder(pipeline_dict, model_file='autoencoder.h5')

Running autoencoder...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Trained autoencoder read from file.
Encoding images...
Done!


In [14]:
pipeline_dict = decomposition.pca_decomp(pipeline_dict, 'features_psd2d', n_components=10)

Running PCA...
Total explained variance: 0.9546159551596857


In [15]:
pipeline_dict = decomposition.truncated_svd_decomp(pipeline_dict, 'features_wavelets', 10)

Running truncated SVD...
Total explained variance: 0.9059018272049221


In [15]:
pipeline_dict = scaling.scale_features(pipeline_dict, 'features_psd2d_pca')

In [17]:
pipeline_dict = scaling.scale_features(pipeline_dict, 'features_wavelets_trunc_svd')

In [18]:
pipeline_dict = scaling.scale_features(pipeline_dict, 'features_autoencoder')

In [16]:
#scores = anomaly_detection.run_isolation_forest(reduced_wav, normalise_scores=True)
#df = isolation_forest.run_isolation_forest(df, features_column_name='psd2d_features', normalise_scores=True)
pipeline_dict = isolation_forest.run_isolation_forest(pipeline_dict, input_key='features_psd2d_pca_scaled')

Running isolation forest...
Done!


In [20]:
pipeline_dict = isolation_forest.run_isolation_forest(pipeline_dict, input_key='features_wavelets_trunc_svd_scaled')

Running isolation forest...
Done!


In [21]:
pipeline_dict = isolation_forest.run_isolation_forest(pipeline_dict, input_key='features_autoencoder_scaled')

Running isolation forest...
Done!


In [17]:
pipeline_dict = human_loop_learning.convert_anomaly_score(pipeline_dict, 
                                                          input_column='iforest_score_features_psd2d_pca_scaled', 
                                                          convert_integer=True)

In [38]:
pipeline_dict = human_loop_learning.convert_anomaly_score(pipeline_dict, 
                                                          input_column='iforest_score_features_wavelets_trunc_svd_scaled', 
                                                          convert_integer=True)

In [39]:
pipeline_dict = human_loop_learning.convert_anomaly_score(pipeline_dict, 
                                                          input_column='iforest_score_features_autoencoder_scaled', 
                                                          convert_integer=True)

In [40]:
pipeline_dict.keys()

dict_keys(['images', 'cutouts', 'metadata', 'features_psd2d', 'features_wavelets', 'features_autoencoder', 'features_psd2d_pca', 'features_psd2d_pca_object', 'features_wavelets_trunc_svd', 'features_wavelets_trunc_svd_object', 'features_psd2d_pca_scaled', 'features_wavelets_trunc_svd_scaled', 'features_autoencoder_scaled', 'ml_scores', 'iforest_object_features_psd2d_pca_scaled', 'iforest_object_features_wavelets_trunc_svd_scaled', 'iforest_object_features_autoencoder_scaled'])

In [18]:
ml_df = pipeline_dict['ml_scores']
ml_df.columns

Index(['id', 'iforest_score_features_psd2d_pca_scaled',
       'iforest_score_features_psd2d_pca_scaled_norm'],
      dtype='object')

In [19]:
ts_psd = TSNE(perplexity=100)
ts_psd.fit(pipeline_dict['features_psd2d_pca_scaled'])

TSNE(angle=0.5, early_exaggeration=12.0, init='random', learning_rate=200.0,
     method='barnes_hut', metric='euclidean', min_grad_norm=1e-07,
     n_components=2, n_iter=1000, n_iter_without_progress=300, perplexity=100,
     random_state=None, verbose=0)

In [23]:
plt.figure()
ts_x = ts_psd.embedding_[:,0]
ts_y = ts_psd.embedding_[:,1]
plt.scatter(ts_x, ts_y, c=pipeline_dict['ml_scores'].iforest_score_features_psd2d_pca_scaled_norm,
            s=10, cmap='magma')

FigureCanvasNbAgg()

In [31]:
ts_wave = TSNE(perplexity=100)
ts_wave.fit(pipeline_dict['features_wavelets_trunc_svd_scaled'])

TSNE(angle=0.5, early_exaggeration=12.0, init='random', learning_rate=200.0,
   method='barnes_hut', metric='euclidean', min_grad_norm=1e-07,
   n_components=2, n_iter=1000, n_iter_without_progress=300,
   perplexity=100, random_state=None, verbose=0)

In [32]:
plt.figure()
ts_x = ts_wave.embedding_[:,0]
ts_y = ts_wave.embedding_[:,1]
plt.scatter(ts_x, ts_y, c=pipeline_dict['ml_scores'].iforest_score_features_wavelets_trunc_svd_scaled_norm,
            s=10, cmap='magma')

FigureCanvasNbAgg()

In [33]:
ts_autoenc = TSNE(perplexity=100)
ts_autoenc.fit(pipeline_dict['features_autoencoder_scaled'])

TSNE(angle=0.5, early_exaggeration=12.0, init='random', learning_rate=200.0,
   method='barnes_hut', metric='euclidean', min_grad_norm=1e-07,
   n_components=2, n_iter=1000, n_iter_without_progress=300,
   perplexity=100, random_state=None, verbose=0)

In [41]:
plt.figure()
ts_x = ts_autoenc.embedding_[:,0]
ts_y = ts_autoenc.embedding_[:,1]
plt.scatter(ts_x, ts_y, c=pipeline_dict['ml_scores']['iforest_score_features_autoencoder_scaled_norm'],
            s=10, cmap='magma')
plt.colorbar()

FigureCanvasNbAgg()

In [21]:
sorted_df = ml_df.sort_values('iforest_score_features_psd2d_pca_scaled_norm', ascending=False)
#sorted_df = ml_df.sort_values('iforest_score_features_wavelets_trunc_svd_scaled_norm', ascending=False)
# sorted_df = ml_df.sort_values('iforest_score_features_autoencoder_scaled_norm', ascending=False)

In [22]:
cycler = utils.ImageCycler(pipeline_dict['cutouts'].loc[list(sorted_df.id),:,:])
cycler.cycle()

FigureCanvasNbAgg()

In [64]:
ids = list(ml_df.id[(ts_x>20)&(ts_y<-5)])

In [65]:
short_df = ml_df.loc[ids]
short_df.sort_values('iforest_score_features_autoencoder_scaled_norm', inplace=True, ascending=False)

In [66]:
cycler = utils.ImageCycler(pipeline_dict['cutouts'][list(short_df.id)])
cycler.cycle()

FigureCanvasNbAgg()

In [210]:
#comp = pca_obj.components_[7]
ind = 1
comp = wav_feats[ind]

reshaped_coeffs = feature_extraction.reshape_swt2_coefficients(comp, 2, (128,128))
lev = 0
plt.figure()
plt.subplot(221)
plt.imshow(reshaped_coeffs[lev][0])

plt.subplot(222)
plt.imshow(reshaped_coeffs[lev][1][0])
plt.subplot(223)
plt.imshow(reshaped_coeffs[lev][1][1])
plt.subplot(224)
plt.imshow(reshaped_coeffs[lev][1][2])
plt.tight_layout()

FigureCanvasNbAgg()

In [195]:
plt.close('all')

In [211]:
plt.figure()
plt.plot(reduced_wav[ind])

FigureCanvasNbAgg()

## Test PSD

In [145]:
x = np.linspace(-np.pi,np.pi,600)
X,Y = np.meshgrid(x,x)
x0 = 0
#Z = np.sin(np.sqrt((X-x0)**2+(Y-x0)**2)*8)
Z = np.sin(X*20)
plt.figure()
plt.imshow(Z)

FigureCanvasNbAgg()

In [146]:
nbins = 100
psd_test= feature_extraction.psd_2d(Z, nbins=nbins)
plt.figure()
rwidth = len(x)/2/nbins
plt.plot(np.arange(rwidth, len(x)/2+rwidth, rwidth),psd_test)

FigureCanvasNbAgg()

In [127]:
plt.close('all')

# Test wavelets

In [14]:
wav_feats = wavelet_features.wavelet_decomposition(cutout)

In [15]:
coeffs = wavelet_features.reshape_swt2_coefficients(wav_feats, 2, cutout.shape)

In [16]:
reconstruction = pywt.iswt2(coeffs, 'sym2')

In [17]:
plt.figure()
plt.subplot(1,2,1)
plt.imshow(cutout)
plt.subplot(1,2,2)
plt.imshow(reconstruction)

FigureCanvasNbAgg()

# Scratch

In [10]:
from astronomaly.data_management import image_reader

In [16]:
importlib.reload(image_reader)

<module 'astronomaly.data_management.image_reader' from '/home/michelle/Project/Anomaly/astronomaly/astronomaly/data_management/image_reader.py'>

In [18]:
pth = '/home/michelle/BigData/Anomaly'
output_dict = image_reader.read_images(pth)

Reading image data from /home/michelle/BigData/Anomaly/ds9_slice1.fits...
Done!


In [20]:
output_dict['images']

In [44]:
importlib.reload(image_preprocessing)

<module 'astronomaly.preprocessing.image_preprocessing' from '/home/michelle/Project/Anomaly/astronomaly/astronomaly/preprocessing/image_preprocessing.py'>

In [45]:
output_dict = image_preprocessing.generate_cutouts(output_dict)


Generating cutouts...
Done!


In [47]:
df = output_dict['data']

In [52]:
cutouts = output_dict['cutouts']

In [58]:
x = xarray.DataArray(cutouts, coords = {'id':df.id}, dims=['id','dim_1','dim_2'])

In [63]:
x[0].shape

(128, 128)

In [67]:
shp = (64.5,128)

In [68]:
min(shp)//2

32.0

In [ ]:
xarray.full_like()